# Lab 2-1. Logistic Regression

*   Regression: input data $x$를 output data $y$로 매핑하는 모델 $f$를 찾는것.

*   Logistic Regression: 모델 $f$가 linear model의 output을 logistc function을 이용해서 0~1사이의 확률값을 예측함, 즉 $$f(\textbf{x})=\sigma(\textbf{w}^T\textbf{x}+b)=\dfrac{1}{1+e^{-(\textbf{w}^T\textbf{x}+b)}}$$

*   Loss function
$$\text{Loss}(\textbf{w})=-\dfrac{1}{N}\sum y\text{log}(f(\textbf{x}))+(1-y)\text{log}(1-f(\textbf{x})) $$   

*    Gradient Descent
$$\textbf{w}=\textbf{w}-\alpha \dfrac{\partial}{\partial\textbf{w}}\text{Loss}(\textbf{w})$$

#### Import torch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Logistic Regression with Toy Training Data

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [ ]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
print(x_train.shape)
print(y_train.shape)

#### Compute the Hypothesis
$$f(\textbf{x})=\sigma(\textbf{w}^T\textbf{x}+b)=\dfrac{1}{1+e^{-(\textbf{w}^T\textbf{x}+b)}}$$


In [ ]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

In [ ]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [ ]:
print(hypothesis)
print(hypothesis.shape)

`torch.sigmoid()` 함수를 쓸 수도 있음


In [ ]:
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

In [ ]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [ ]:
print(hypothesis)
print(hypothesis.shape)

#### Computing the Loss Function (Low-level)

$$\text{Loss}(\textbf{w})=-\dfrac{1}{N}\sum y\text{log}(f(\textbf{x}))+(1-y)\text{log}(1-f(\textbf{x})) $$   

 `hypothesis` and `y_train` 사이의 차이를 측정하고 싶음.

In [ ]:
print(hypothesis)
print(y_train)

하나의 데이터에 대해서 다음과 같이 계산 가능

In [ ]:
-(y_train[0] * torch.log(hypothesis[0]) + 
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

전체 데이터에 대해서는 다음과 같이 간단하게 계산 가능

In [ ]:
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

`.mean()` 을 사용하여 평균을 취함

In [ ]:
cost = losses.mean()
print(cost)

#### Computing the Cost Function with `F.binary_cross_entropy`

실제로는 binary classification은 자주 쓰이기 때문에 pytorch에 `F.binary_cross_entropy` 라는 함수가 구현돼있음

(지난번 실습의 `torch.nn.MSELoss`처럼)

In [ ]:
F.binary_cross_entropy(hypothesis, y_train)

#### Training with Low-level Binary Cross Entropy Loss

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

#### Training with `F.binary_cross_entropy`

In [ ]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

## Logistic Regression with Real Data

### Load File from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
diabetes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/diabetes.csv')
print(diabetes.columns)
diabetes.head()

In [ ]:
type(diabetes)

In [ ]:
print("dimension of diabetes data: {}".format(diabetes.shape))

### Split and prepare the dataset

In [ ]:
train = diabetes[:650]
test = diabetes[650:]

In [ ]:
print(train.groupby('Outcome').size())

imbalanced dataset임을 확인 가능

In [ ]:
x_train = np.asarray(train.drop('Outcome',1))
y_train = np.asarray(train['Outcome'])
x_test = np.asarray(test.drop('Outcome',1))
y_test = np.asarray(test['Outcome'])

input을 normalize 해준다.

In [ ]:
means = np.mean(x_train, axis=0)
stds = np.std(x_train, axis=0)
x_train = (x_train - means)/stds
x_test = (x_test - means)/stds

In [ ]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train).unsqueeze(-1)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test).unsqueeze(-1)

### Training with F.binary_cross_entropy

In [ ]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 f(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

### Test Accuracy

In [ ]:
W, b

In [ ]:
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.float() == y_test
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the test set.'.format(accuracy * 100))

## (Optional) Balancing the training set

In [ ]:
x_train = np.asarray(train.drop('Outcome',1))
y_train = np.asarray(train['Outcome'])
x_test = np.asarray(test.drop('Outcome',1))
y_test = np.asarray(test['Outcome'])
means = np.mean(x_train, axis=0)
stds = np.std(x_train, axis=0)
x_train = (x_train - means)/stds
x_test = (x_test - means)/stds

In [ ]:
x_train_pos = x_train[y_train == 0]
x_train_neg = x_train[y_train == 1]
y_train_pos = y_train[y_train == 0]
y_train_neg = y_train[y_train == 1]

In [ ]:
x_train_pos.shape, x_train_neg.shape

In [ ]:
ids = np.arange(len(x_train_neg))
choices = np.random.choice(ids, len(x_train_pos))
x_train_neg = x_train_neg[choices]
y_train_neg = y_train_neg[choices]

In [ ]:
x_train = np.concatenate([x_train_pos, x_train_neg], axis=0)
y_train = np.concatenate([y_train_pos, y_train_neg], axis=0)

order = np.arange(len(x_train))
np.random.shuffle(order)
x_train = x_train[order]
y_train = y_train[order]

In [ ]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train).unsqueeze(-1)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test).unsqueeze(-1)

### Training with F.binary_cross_entropy

In [ ]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 f(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

### Test Accuracy

In [ ]:
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.float() == y_test
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))